<a href="https://colab.research.google.com/github/catalyst-team/neuro/blob/master/examples/Neuro_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuro UNet/Meshnet Tutorial

This is a reimplementation of [An (almost) instant brain atlas segmentation for
large-scale studies](https://arxiv.org/pdf/1711.00457.pdf) using the publicly available Mindboggle Dataset.  Given only 70 volumes for training (typically 700+) and very minimal augmentations we can achieve a 0.6688 Mean Dice Score for a brain with a majority vote classification.  With more brains and normalizing the T1 scans we could likely perform better


Authors: [Kevin Wang](https://github.com/ssktotoro/), [Alex Fedorov](https://github.com/Entodi/), [Sergey Kolesnikov](https://github.com/Scitator)

[![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)](https://github.com/catalyst-team/catalyst)

### Colab setup

First of all, do not forget to change the runtime type to GPU. <br/>
To do so click `Runtime` -> `Change runtime type` -> Select `\"Python 3\"` and `\"GPU\"` -> click `Save`. <br/>
After that you can click `Runtime` -> `Run all` and watch the tutorial.

## Requirements

Download and install the latest versions of catalyst and other libraries required for this tutorial.

In [ ]:
%%bash 
# git clone https://github.com/catalyst-team/neuro.git
pip install -r neuro/requirements/requirements.txt


In [ ]:
import torch                                                                                                                                                                                              
from tqdm import tqdm                                                                                                                                                                                     
import numpy as np                                                                                                                                                                                        
import nibabel as nib                                                                                                                                                                                     
import collections                                                                                                                                                                                        
from collections import OrderedDict                                                                                                                                                                       
                                                                                                                                                                                                          
import catalyst                                                                                                                                                                                           
import pandas as pd  
import os                                                                                                                                                                                     
                                                                                                                                                                                                          
from catalyst.contrib.utils.pandas import dataframe_to_list                                                                                                                                               
from torch.utils.data import SequentialSampler                                                                                                                                                            
from torch.utils.data import DataLoader                                                                                                                                                                   
from catalyst.data import ReaderCompose                                                                                                                                                                   
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR                                                                                                                                        
from catalyst.callbacks import CheckpointCallback                                                                                                                                                         
from torch.nn import functional as F                                                                                                                                                                      
from typing import List
from catalyst import utils                                                                                                                                                                                   
                                                                                                                                                                                                                                                                                                                                                                                    
from catalyst import metrics                                                                                                                                                                              
from catalyst.data import BatchPrefetchLoaderWrapper                                                                                                                                                      
from catalyst.dl import Runner, LRFinder                                                                                                                                                                  
                                                                                                                                                                                                          
from catalyst.metrics.functional._segmentation import dice


print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU

SEED = 42
utils.set_global_seed(SEED)
utils.prepare_cudnn(deterministic=True)

# Dataset

We'll be using the Mindboggle 101 dataset for a multiclass 3d segmentation task.
The dataset can be downloaded off osf with the following command from osfclient after you register with osf.

`osf -p 9ahyp clone .`

Otherwise you can download it using a Catalyst utility `download-gdrive` which downloads a version from the Catalyst Google Drive

`usage: download-gdrive {FILE_ID} {FILENAME}`

In [ ]:
cd neuro

In [ ]:
%%bash
mkdir Mindboggle_data 
mkdir -p data/Mindboggle_101/
osf -p 9ahyp clone Mindboggle_data/
cp -r Mindboggle_data/osfstorage/Mindboggle101_volumes/ data/Mindboggle_101/
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i tar zxvf {} -C data/Mindboggle_101
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i rm {}

Run the prepare data script that limits the labels to the DKT cortical labels (31 labels).  We can use of course use more labels.

`usage: python ../neuro/scripts/prepare_data.py ../data/Mindboggle_101 {N_labels)`

In [ ]:
%%bash 

python neuro/scripts/prepare_data.py data/Mindboggle_101/ 31

### **Create the relevant Dataloaders for the specified train, validation, and inference BrainDatasets.**

BrainDatasets comprise of T1 scans + the prepared limited labels.

Training/ Validation batches: To avoid sampling outside the brain area we randomly sampled NxNxN subvolumes from a gaussian distribution with the mean set in the center of the volume and the diagonal covariance set at 50.  This helps avoid class imbalance that would lead the model to simpley prefer the background class.

Inference batches: All non-overlapping NxNxN Subvolumes across the existing volume space with their corresponding labels + randomly sampled NxNxN subvolumes from a the same gaussian distribution accross volume space as above until the required number of subvolume is reached.

More detail can be found in brain_dataset.py and generator_coords.py  

In [ ]:
cd neuro/

In [ ]:
from neuro.brain_dataset import BrainDataset
from neuro.reader import NiftiFixedVolumeReader, NiftiReader
from neuro.model import MeshNet, UNet

In [ ]:
def get_loaders(
  random_state: int,
  volume_shape: List[int],
  subvolume_shape: List[int],
  in_csv_train: str = None,
  in_csv_valid: str = None,
  in_csv_infer: str = None,
  batch_size: int = 16,
  num_workers: int = 10,
) -> dict:

  datasets = {}
  open_fn = ReaderCompose(
      [
          NiftiFixedVolumeReader(input_key="images", output_key="images"),
          NiftiReader(input_key="nii_labels", output_key="targets"),

      ]
  )

  for mode, source in zip(("train", "validation", "infer"),
                          (in_csv_train, in_csv_valid, in_csv_infer)):
      if mode == "infer":
          n_subvolumes = 512
      else:
          n_subvolumes = 128

      if source is not None and len(source) > 0:
          dataset = BrainDataset(
              list_data=dataframe_to_list(pd.read_csv(source)),
              list_shape=volume_shape,
              list_sub_shape=subvolume_shape,
              open_fn=open_fn,
              n_subvolumes=n_subvolumes,
              mode=mode,
              input_key="images",
              output_key="targets",
          )

      datasets[mode] = {"dataset": dataset}

  def worker_init_fn(worker_id):
      np.random.seed(np.random.get_state()[1][0] + worker_id)


  train_loader = DataLoader(dataset=datasets['train']['dataset'], batch_size=batch_size,
                            shuffle=True, worker_init_fn=worker_init_fn,
                            num_workers=2, pin_memory=True)
  valid_loader = DataLoader(dataset=datasets['validation']['dataset'],
                            shuffle=True, worker_init_fn=worker_init_fn,
                            batch_size=batch_size,
                            num_workers=2, pin_memory=True,drop_last=True)
  test_loader = DataLoader(dataset=datasets['infer']['dataset'],
                           batch_size=batch_size, worker_init_fn=worker_init_fn,
                           num_workers=2, pin_memory=True,drop_last=True)
  train_loaders = collections.OrderedDict()
  infer_loaders = collections.OrderedDict()
  train_loaders["train"] = BatchPrefetchLoaderWrapper(train_loader)
  train_loaders["valid"] = BatchPrefetchLoaderWrapper(valid_loader)
  infer_loaders['infer'] = BatchPrefetchLoaderWrapper(test_loader)

  return train_loaders, infer_loaders

In [ ]:
cd ../

In [ ]:
volume_shape = [256, 256, 256]
subvolume_shape = [38, 38, 38]
train_loaders, infer_loaders = get_loaders(
    0, volume_shape, subvolume_shape,
    "./data/dataset_train.csv",
    "./data/dataset_valid.csv",
    "./data/dataset_infer.csv"
)

# Model Training

We'll train the model 30 epochs.

* Scheduler: Adam with a One Cycle Learning Rate with a Max Learning Rate of .02
* Batch Metric: DICE
* Loss: CrossEntropyLoss
* Logger: Tensorboard
* CheckpointerCallback

For training and validation we sample the volume with subvolumes specified in our Dataset 

In [ ]:
class CustomRunner(Runner):

    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        """Returns the loaders for a given stage."""
        self._loaders = self._loaders
        return self._loaders

    def predict_batch(self, batch):
        # model inference step
        batch = batch[0]
        return self.model(batch['images'].float().to(self.device)), batch['coords']

    def on_loader_start(self, runner):
        super().on_loader_start(runner)
        self.meters = {
          key: metrics.AdditiveValueMetric(compute_on_call=False)
          for key in ["loss", "macro_dice"]
        }

    def handle_batch(self, batch):
        # model train/valid step
        batch = batch[0]
        x, y = batch['images'].float(), batch['targets']

        if self.is_train_loader:
          self.optimizer.zero_grad()

        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)

        if self.is_train_loader:
          loss.backward()
          self.optimizer.step()
          scheduler.step()

        one_hot_targets = (
          torch.nn.functional.one_hot(y, 31)
          .permute(0, 4, 1, 2, 3)
          .cuda()
        )

        logits_softmax = F.softmax(y_hat)
        macro_dice = dice(logits_softmax, one_hot_targets, mode='macro')

        self.batch_metrics.update({"loss": loss, 'macro_dice': macro_dice})
        for key in ["loss", "macro_dice"]:
          self.meters[key].update(self.batch_metrics[key].item(), self.batch_size)

    def on_loader_end(self, runner):
        for key in ["loss", "macro_dice"]:
            self.loader_metrics[key] = self.meters[key].compute()[0]
        super().on_loader_end(runner)
                                           

In [ ]:
n_classes = 31
n_epochs = 30
meshnet = MeshNet(n_channels=1, n_classes=n_classes)

logdir = "logs/meshnet_mindboggle"

optimizer = torch.optim.Adam(meshnet.parameters(), lr=0.02)


scheduler = OneCycleLR(optimizer, max_lr=.02,
                 epochs=n_epochs, steps_per_epoch=len(train_loaders['train']))

runner = CustomRunner()
runner.train(
    model=meshnet,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_loaders,
    num_epochs=n_epochs,
    logdir=logdir,
    callbacks=[CheckpointCallback(logdir=logdir)],
    verbose=True
)

# Model Evaluation

For every brain volume we implement a majority vote for every voxel and use  that to compute a Dice score.


The initial volume is segmented into
a regular grid of subvolumes partitioning the whole volume.
These volumes ensure a prediction for each voxel. Then we
sample overlapping volumes from the brain region until N
subvolumes (512 in this case) are achieved for prediction.  



In [ ]:
def voxel_majority_predict_from_subvolumes(loader, n_classes, segmentations):
    if segmentations is None:
        for subject in range(loader.dataset.subjects):
            segmentations[subject] = torch.zeros(
                tuple(np.insert(loader.volume_shape, 0, n_classes)),
                dtype=torch.uint8).cpu()

    prediction_n = 0
    for inference in tqdm(runner.predict_loader(loader=loader)):
        coords = inference[1].cpu()
        _, predicted = torch.max(F.log_softmax(inference[0].cpu(), dim=1), 1)
        for j in range(predicted.shape[0]):
            c_j = coords[j][0]
            subj_id = prediction_n // loader.dataset.n_subvolumes
            for c in range(n_classes):
                segmentations[subj_id][c, c_j[0, 0]:c_j[0, 1],
                                       c_j[1, 0]:c_j[1, 1],
                                       c_j[2, 0]:c_j[2, 1]] += (predicted[j] == c)
            prediction_n += 1

    for i in segmentations.keys():
        segmentations[i] = torch.max(segmentations[i], 0)[1]
    return segmentations

In [ ]:
segmentations = {}
for subject in range(infer_loaders['infer'].dataset.subjects):
    segmentations[subject] = torch.zeros(tuple(np.insert(volume_shape, 0, n_classes)), dtype=torch.uint8)

In [ ]:
segmentations = voxel_majority_predict_from_subvolumes(infer_loaders['infer'],
                                                     n_classes, segmentations)
subject_metrics = []
for subject, subject_data in enumerate(tqdm(infer_loaders['infer'].dataset.data)):
    seg_labels = nib.load(subject_data['nii_labels']).get_fdata()
    segmentation_labels = torch.nn.functional.one_hot(
        torch.from_numpy(seg_labels).to(torch.int64), n_classes)

    inference_dice = dice(
        torch.nn.functional.one_hot(segmentations[subject], n_classes).permute(0, 3, 1, 2),
        segmentation_labels.permute(0, 3, 1, 2)
    ).detach().numpy()
    macro_inference_dice = dice(
        torch.nn.functional.one_hot(segmentations[subject], n_classes).permute(0, 3, 1, 2),
        segmentation_labels.permute(0, 3, 1, 2), mode='macro'
    ).detach().numpy()
    subject_metrics.append((inference_dice, macro_inference_dice))

per_class_df = pd.DataFrame([metric[0] for metric in subject_metrics])
macro_df = pd.DataFrame([metric[1] for metric in subject_metrics])
print(per_class_df, macro_df)
print(macro_df.mean())